<a href="https://colab.research.google.com/github/sharik31/SQL-Generator/blob/main/Spider_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft bitsandbytes accelerate
!pip uninstall -y transformers accelerate
!pip install -U "transformers>=4.41.0" "accelerate>=0.33.0" "datasets>=2.20.0" evaluate peft bitsandbytes
!pip install -U transformers datasets peft accelerate trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.5 MB/s eta 0:00:00
Found existing installation: transformers 4.55.2
Uninstalling transformers-4.55.2:
  Successfully uninstalled transformers-4.55.2
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 11.3 MB/s eta 0:00:00


In [2]:
import os

os.makedirs("/content/adapter_checkpoint", exist_ok=True)


In [6]:
import shutil

shutil.move("/content/adapter_config.json", "/content/adapter_checkpoint/adapter_config.json")
shutil.move("/content/adapter_model.safetensors", "/content/adapter_checkpoint/adapter_model.safetensors")


'/content/adapter_checkpoint/adapter_model.safetensors'

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from peft import PeftModel


In [8]:
base_model_name = "t5-small"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)


adapter_dir = "/content/adapter_checkpoint"
model = PeftModel.from_pretrained(base_model, adapter_dir, is_trainable=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [9]:
trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print("Trainable parameters:", trainable_params)
print("Total trainable params:", len(trainable_params))

Trainable parameters: ['base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight', 'base_model.model.encoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_A.default.weight', 'base_model.model.encoder.block.1.layer.0.SelfAttention.v.lora_B.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_A.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.q.lora_B.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.v.lora_A.default.weight', 'base_model.model.encoder.block.2.layer.0.SelfAttention.v.lora_B.default.we

In [10]:
from datasets import load_dataset
dataset = load_dataset(
    "csv",
    data_files={
        "train": "/content/spider_train_with_separate_schema_column.csv",
        "validation": "/content/spider_val_with_separate_schema_column.csv",
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [11]:
def preprocess_spider(examples):
    merged_inputs = [
        f"{q} [SCHEMA] {s}" for q, s in zip(examples["input"], examples["schema"])
    ]
    model_inputs = tokenizer(merged_inputs, truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=256)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_spider_dataset = dataset.map(preprocess_spider, batched=True)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [12]:
print(tokenized_spider_dataset)


DatasetDict({
    train: Dataset({
        features: ['db_id', 'input', 'output', 'schema', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'input', 'output', 'schema', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1034
    })
})


In [13]:

for i in range(3):
    row = dataset["train"][i]
    print(f"Example {i+1}:")
    print("Natural Language Question:", row["input"])
    print("Schema:", row["schema"])
    print("Target SQL:", row["output"])
    print("-" * 50)


Example 1:
Natural Language Question: translate to SQL: How many heads of the departments are older than 56 ?
Schema: Table department (Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees) Table head (head_ID, name, born_state, age) Table management (department_ID, head_ID, temporary_acting)
Target SQL: SELECT count(*) FROM head WHERE age  >  56
--------------------------------------------------
Example 2:
Natural Language Question: translate to SQL: List the name, born state and age of the heads of departments ordered by age.
Schema: Table department (Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees) Table head (head_ID, name, born_state, age) Table management (department_ID, head_ID, temporary_acting)
Target SQL: SELECT name ,  born_state ,  age FROM head ORDER BY age
--------------------------------------------------
Example 3:
Natural Language Question: translate to SQL: List the creation year, name and budget of each department.
Schem

In [14]:
for i in range(3):
    row = tokenized_spider_dataset["train"][i]
    input_text = tokenizer.decode(row["input_ids"], skip_special_tokens=True)
    label_ids = [id for id in row["labels"] if id != -100]
    output_text = tokenizer.decode(label_ids, skip_special_tokens=True)
    print(f"Example {i+1} (after tokenization):")
    print("Model Input:", input_text)
    print("Model Label (SQL):", output_text)
    print("-" * 50)


Example 1 (after tokenization):
Model Input: translate to SQL: How many heads of the departments are older than 56? [SCHEMA] Table department (Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees) Table head (head_ID, name, born_state, age) Table management (department_ID, head_ID, temporary_acting)
Model Label (SQL): SELECT count(*) FROM head WHERE age > 56
--------------------------------------------------
Example 2 (after tokenization):
Model Input: translate to SQL: List the name, born state and age of the heads of departments ordered by age. [SCHEMA] Table department (Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees) Table head (head_ID, name, born_state, age) Table management (department_ID, head_ID, temporary_acting)
Model Label (SQL): SELECT name, born_state, age FROM head ORDER BY age
--------------------------------------------------
Example 3 (after tokenization):
Model Input: translate to SQL: List the creation year, name and bu

In [15]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./spider_results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-3,
    num_train_epochs=4,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./spider_logs"
)

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_spider_dataset["train"],
    eval_dataset=tokenized_spider_dataset["validation"],
    processing_class=tokenizer,
    data_collator=data_collator
)


In [19]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shujasharik31 (shujasharik31-jamia-millia-islamia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.363300
1000,0.242500
1500,0.194400
2000,0.171900
2500,0.161100
3000,0.145700
3500,0.140100
4000,0.137700
4500,0.131500
5000,0.122800


TrainOutput(global_step=14000, training_loss=0.12990992655072894, metrics={'train_runtime': 1643.7024, 'train_samples_per_second': 17.035, 'train_steps_per_second': 8.517, 'total_flos': 1907468795904000.0, 'train_loss': 0.12990992655072894, 'epoch': 4.0})

In [20]:
from safetensors.torch import safe_open
with safe_open("/content/adapter_checkpoint/adapter_model.safetensors", framework="pt") as f:
    print(list(f.keys()))


['base_model.model.decoder.block.0.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.decoder.block.0.layer.0.SelfAttention.q.lora_B.weight', 'base_model.model.decoder.block.0.layer.0.SelfAttention.v.lora_A.weight', 'base_model.model.decoder.block.0.layer.0.SelfAttention.v.lora_B.weight', 'base_model.model.decoder.block.0.layer.1.EncDecAttention.q.lora_A.weight', 'base_model.model.decoder.block.0.layer.1.EncDecAttention.q.lora_B.weight', 'base_model.model.decoder.block.0.layer.1.EncDecAttention.v.lora_A.weight', 'base_model.model.decoder.block.0.layer.1.EncDecAttention.v.lora_B.weight', 'base_model.model.decoder.block.1.layer.0.SelfAttention.q.lora_A.weight', 'base_model.model.decoder.block.1.layer.0.SelfAttention.q.lora_B.weight', 'base_model.model.decoder.block.1.layer.0.SelfAttention.v.lora_A.weight', 'base_model.model.decoder.block.1.layer.0.SelfAttention.v.lora_B.weight', 'base_model.model.decoder.block.1.layer.1.EncDecAttention.q.lora_A.weight', 'base_model.model.decoder.b

In [21]:
model.print_trainable_parameters()


trainable params: 294,912 || all params: 60,801,536 || trainable%: 0.4850


In [22]:
model.save_pretrained("final_sql_model")

In [29]:
test_query = "translate to SQL: List the names and countries of singers who are older than 30.?"
input_string = f"{test_query}"

inputs = tokenizer(input_string, return_tensors="pt", truncation=True, padding=True, max_length=256)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_length=128)
generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Predicted SQL query:\n", generated_sql)


Predicted SQL query:
 SELECT name & country FROM singers WHERE age > 30


In [24]:

results = trainer.evaluate()
print(results)


{'eval_loss': 0.10986328125, 'eval_runtime': 34.7932, 'eval_samples_per_second': 29.718, 'eval_steps_per_second': 14.859, 'epoch': 4.0}


In [25]:

!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=abe498721bb9c56bc95f620afc51fef71b4b17adb55017ac308ba77c2b168a6e
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [26]:
from evaluate import load

rouge = load("rouge")

predictions = []
references = []

for example in tokenized_spider_dataset["validation"]:
    input_text = tokenizer.decode(example["input_ids"], skip_special_tokens=True)

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(**inputs, max_length=128)
    pred_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
   ## decoding ref sql
    reference_sql = tokenizer.decode([id for id in example["labels"] if id != -100], skip_special_tokens=True)
    predictions.append(pred_sql)
    references.append(reference_sql)

In [27]:
rouge_scores = rouge.compute(predictions=predictions, references=references)
print("ROUGE scores:", rouge_scores)


ROUGE scores: {'rouge1': np.float64(0.6859988633093634), 'rouge2': np.float64(0.5248568984823355), 'rougeL': np.float64(0.6666748933898166), 'rougeLsum': np.float64(0.6664125324795367)}


In [32]:
!pip install huggingface_hub

In [33]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `sql` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credentia

In [36]:
from huggingface_hub import HfApi

api = HfApi()

repo_id = "Shujashark/llm-sql-t5-small-lora-adapter"
local_folder = "final_sql_model"

api.upload_folder(
    folder_path=local_folder,
    repo_id=repo_id,
    repo_type="model"
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...sql_model/adapter_model.safetensors:  47%|####6     |  559kB / 1.19MB            

CommitInfo(commit_url='https://huggingface.co/Shujashark/llm-sql-t5-small-lora-adapter/commit/68d9bb7eb4334968914e81c5ad13469aecdafe9d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='68d9bb7eb4334968914e81c5ad13469aecdafe9d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Shujashark/llm-sql-t5-small-lora-adapter', endpoint='https://huggingface.co', repo_type='model', repo_id='Shujashark/llm-sql-t5-small-lora-adapter'), pr_revision=None, pr_num=None)